In [62]:
import sys

from sklearn.model_selection import train_test_split, KFold


data_path = '/Users/tyamgin/Projects/mlbootcamp/vkcup2021_2/data/'

In [74]:
%run /Users/tyamgin/Projects/mlbootcamp/vkcup2021_2/lib.py

/usr/local/lib/python3.9/site-packages/sklearn/decomposition/_truncated_svd.py:205: RuntimeWarning: invalid value encountered in true_divide
  self.explained_variance_ratio_ = exp_var / full_var
/usr/local/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.9/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1111: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/tyamgin/Projects/mlbootcamp/vkcup2021_2/lib.py:190: RuntimeWarning: Mean of empty slice
  np.nanmean(self.age_by_uid2[friends2.get(uid, [])])


In [75]:
train = Data()
train.read(data_path, 'train')

In [76]:
def cross_validation(model, data, n_folds=5, n_iters=5, seed=2707, verbose=1):
    scores = []
    user_ids = data.edu['uid'].unique()
    
    for iter in range(n_iters):
        kf = KFold(n_splits=n_folds, random_state=seed+iter, shuffle=True)
        print(kf)
        fold = 0
        for train_index, test_index in kf.split(user_ids):
            fold += 1
            if verbose >= 2:
                print('Prepare data: %s' % datetime.datetime.now())
                
            data_train = data.get(user_ids[train_index])
            data_test = data.get(user_ids[test_index])
            test_age = data_test.edu['age'].values
            data_test.edu.drop(['age'], 1, inplace=True)

            if verbose >= 2:
                print('Fit: %s' % datetime.datetime.now())
            model.prepare(data_train, data_test)
            model.fit(data_train)
            if verbose >= 2:
                print('Predict: %s' % datetime.datetime.now())

            assert 'age' not in data_test.edu.columns
            pred = model.predict(data_test)
            if verbose >= 2:
                print('Auc: %s' % datetime.datetime.now())
            score = rmse(test_age, pred['res'].values)
            scores.append(score)

            #if verbose >= 1:
            print('%2d -%2d : %1.4f, mean=%1.4f' % (iter, fold, score, np.mean(scores)))
    return np.mean(scores)

In [77]:
%%time
seed_everything(12322)

grid = expand_grid({
    'boosting_type': ['gbdt'],
    'min_data_in_leaf': [25],
    'lambda_l2': [0.06],
    'num_leaves': [20],
    'learning_rate': [0.035],
    'feature_fraction': [0.8],
    'bagging_fraction': [0.9],
    'bagging_freq': [6],
    'num_boost_round': [400],
    'group_embeddings_n_components': [5],
    'group_embeddings_n_iter': [44],
    
    'epochs': [100],
    'n1': [170],
    'n2': [150],
    'dropout': [0.1],
})
min_rmse = 1e10
best_params = None
for i, r in grid.iterrows():
    model = MeanModel([
        LgbModel({
            'boosting_type': r.boosting_type,
            'min_data_in_leaf': r.min_data_in_leaf,
            'lambda_l2': r.lambda_l2,
            'num_leaves': r.num_leaves,
            'learning_rate': r.learning_rate,
            'feature_fraction': r.feature_fraction,
            'bagging_fraction': r.bagging_fraction,
            'bagging_freq': r.bagging_freq,
            'num_boost_round': r.num_boost_round,
            'group_embeddings_n_components': r.group_embeddings_n_components,
            'group_embeddings_n_iter': r.group_embeddings_n_iter,
        }),
        KerasModel({
            'epochs': r.epochs,
            'verbose': 0,
            'n1': r.n1,
            'n2': r.n2,
            'dropout': r.dropout,
            #'group_embeddings_n_components': r.group_embeddings_n_components,
            #'group_embeddings_n_iter': r.group_embeddings_n_iter,
        }),
    ], coefs=(0.7, 0.3))
    rr = cross_validation(model, train, n_folds=5, n_iters=1, verbose=0)
    if rr < min_rmse:
        min_rmse = rr
        best_params = r.copy()
    print(r)
    print(rr)
        

KFold(n_splits=5, random_state=2707, shuffle=True)


/usr/local/lib/python3.9/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/usr/local/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1113: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(a, func=_nanmedian, axis=axis, out=out,
/usr/local/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.9/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1111: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/t

Starting train: 2021-07-30 00:18:18.623727
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000522 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3002
[LightGBM] [Info] Number of data points in the train set: 23575, number of used features: 23
[LightGBM] [Info] Start training from score 35.017179
 0 - 1 : 8.4071, mean=8.4071


/usr/local/lib/python3.9/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/usr/local/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1113: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(a, func=_nanmedian, axis=axis, out=out,


KeyboardInterrupt: 

In [ ]:
 0 - 1 : 8.4005, mean=8.4005
 0 - 2 : 8.4647, mean=8.4326
 0 - 3 : 8.2342, mean=8.3665
 0 - 4 : 8.2489, mean=8.3371
 0 - 5 : 8.5243, mean=8.3745

Wall time: 5min 2s

In [21]:
print(min_rmse)
print(best_params)

8.37452339846261
boosting_type                     gbdt
min_data_in_leaf                    25
lambda_l2                         0.06
num_leaves                          20
learning_rate                    0.035
feature_fraction                   0.8
bagging_fraction                   0.9
bagging_freq                         6
num_boost_round                    400
group_embeddings_n_components        5
group_embeddings_n_iter             44
epochs                             100
n1                                 170
n2                                 150
dropout                            0.1
Name: 0, dtype: object


In [22]:
print(min_rmse)
print(best_params)

8.37452339846261
boosting_type                     gbdt
min_data_in_leaf                    25
lambda_l2                         0.06
num_leaves                          20
learning_rate                    0.035
feature_fraction                   0.8
bagging_fraction                   0.9
bagging_freq                         6
num_boost_round                    400
group_embeddings_n_components        5
group_embeddings_n_iter             44
epochs                             100
n1                                 170
n2                                 150
dropout                            0.1
Name: 0, dtype: object


In [ ]:
#0 - 1 : 8.4259, mean=8.4259
#0 - 2 : 8.4633, mean=8.4446
#0 - 3 : 8.2659, mean=8.3850
#0 - 4 : 8.2975, mean=8.3631
#0 - 5 : 8.5725, mean=8.4050

In [602]:
print(min_rmse)
print(best_params)

8.405022744017554
boosting_type        gbdt
min_data_in_leaf       25
lambda_l2            0.06
num_leaves             20
learning_rate       0.035
feature_fraction      0.8
bagging_fraction      0.9
bagging_freq            6
num_boost_round       400
Name: 0, dtype: object


In [215]:
prod_model = LgbModel({
    'boosting_type': 'gbdt',
    'min_data_in_leaf': 25,
    'lambda_l2': 0.06,
    'num_leaves': 20,
    'learning_rate': 0.035,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.9,
    'bagging_freq': 6,
    'num_boost_round': 400,
})
prod_model.prepare(train, None)
prod_model.fit(train)
prod_model.model.save_model('/Users/tyamgin/Projects/mlbootcamp/vkcup2021_2/models/lgb.txt')
prod_model.predict(train)

/usr/local/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.9/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Starting train: 2021-07-26 22:03:06.492044
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002807 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1186
[LightGBM] [Info] Number of data points in the train set: 29469, number of used features: 15
[LightGBM] [Info] Start training from score 35.029149


,uid,res
0,356,45.293078
1,26356,40.299689
2,97,36.529859
3,319,37.234419
4,30288,32.221299
...,...,...
29464,33331,54.447875
29465,119788,40.710635
29466,97581,36.045508
29467,104038,69.633122


In [58]:
a = train.edu.copy()

In [87]:
[
 np.median([
     train.registered_year_by_uid[fr]
     for fr in train.friends.get(uid, [])
     if fr in train.registered_year_by_uid
 ])
 for uid in a['uid'].values
]

[2009.0,
 2012.0,
 2010.0,
 2011.0,
 2010.0,
 2013.0,
 2013.0,
 2010.0,
 2011.0,
 2010.0,
 2014.0,
 2012.0,
 2011.0,
 2010.0,
 2012.0,
 2019.0,
 2011.0,
 2015.5,
 2010.0,
 2013.0,
 2010.0,
 2011.0,
 2015.0,
 2014.0,
 2011.0,
 nan,
 nan,
 2010.0,
 nan,
 nan,
 2011.0,
 2012.5,
 2011.0,
 2014.0,
 2011.0,
 2011.0,
 2009.0,
 2015.0,
 2012.0,
 2018.0,
 2013.0,
 2014.5,
 2011.0,
 nan,
 2014.0,
 nan,
 2011.0,
 2018.0,
 2009.0,
 nan,
 2014.0,
 2010.0,
 2012.5,
 2008.5,
 2012.0,
 2010.0,
 2009.5,
 nan,
 2013.0,
 2018.0,
 2012.0,
 2014.0,
 2010.0,
 2010.0,
 2016.0,
 2011.0,
 2010.0,
 2009.5,
 2011.0,
 2013.0,
 2014.0,
 2016.5,
 2014.0,
 2010.0,
 2016.0,
 2013.0,
 2015.5,
 2011.0,
 2011.0,
 2011.0,
 2013.5,
 2010.0,
 2018.0,
 2011.0,
 2008.0,
 2014.0,
 nan,
 2010.5,
 2009.5,
 2015.0,
 2015.0,
 2011.0,
 2012.0,
 2013.0,
 2014.0,
 2015.5,
 2016.0,
 2010.0,
 2018.0,
 2011.0,
 nan,
 2017.0,
 2011.0,
 2012.0,
 nan,
 2014.5,
 nan,
 2009.0,
 2014.0,
 2013.0,
 2013.0,
 2012.0,
 2012.0,
 2010.0,
 nan,
 201

In [79]:
np.median

<function numpy.median(a, axis=None, out=None, overwrite_input=False, keepdims=False)>

In [83]:
np.median([])

nan

In [97]:
dir(train)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_prepare_friends',
 '_prepare_groups',
 'edu',
 'friends',
 'get',
 'groups',
 'read',
 'registered_year_by_uid']

In [104]:
'ag1e' in train.edu.columns

False

In [112]:
a = pd.read_csv(os.path.join(data_path, f'train.csv')).drop(['age'], 1)
a.to_csv(os.path.join(data_path, f'test.csv'), header=True, index=False)

In [122]:
train.edu

,uid,school_education,graduation_1,graduation_2,graduation_3,graduation_4,graduation_5,graduation_6,graduation_7,age,registered_year
0,356,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48,2017
1,26356,NaN,NaN,NaN,NaN,NaN,2006.0,NaN,NaN,49,2008
2,97,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35,2011
3,319,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37,2020
4,30288,2008.0,NaN,NaN,NaN,NaN,2013.0,NaN,NaN,30,2008
...,...,...,...,...,...,...,...,...,...,...,...
29464,33331,1987.0,NaN,NaN,NaN,NaN,1991.0,NaN,NaN,51,2008
29465,119788,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21,2013
29466,97581,2002.0,NaN,NaN,NaN,NaN,2007.0,NaN,NaN,36,2010
29467,104038,1969.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,69,2013


In [181]:
list(train.groups.items())[0]

(34673,
 array([  7189,  12606,  25510,  43401,  52467,  59728,  74127,  74134,
        110624, 135972, 148721, 157789, 157912, 161477, 168378, 170127,
        170187, 190191, 213872, 219293, 228244, 240885, 251760, 257141,
        262585, 266111, 276925, 287893, 298636, 298638, 300538, 306743,
        335045, 336892, 338667, 342334, 342343, 344140]))

In [292]:
np.median([])

nan

In [295]:
np.median([np.nan, np.nan, 1])

nan

In [422]:
train.edu.uid

0           356
1         26356
2            97
3           319
4         30288
          ...  
29464     33331
29465    119788
29466     97581
29467    104038
29468     95258
Name: uid, Length: 29469, dtype: int64

In [496]:
coo_matrix(([-5,5], ([2,1], [1,2])), shape=(4, 4)).toarray()

array([[ 0,  0,  0,  0],
       [ 0,  0,  5,  0],
       [ 0, -5,  0,  0],
       [ 0,  0,  0,  0]])

In [524]:
x = []
y = []
uids = []
for i, uid in enumerate(train.edu.uid):
    for gid in train.groups.get(uid, []):
        x.append(i)
        y.append(gid)
    uids.append(uid)
mat = coo_matrix((np.repeat(1, len(x)), (x, y)), shape=(max(x) + 1, max(y) + 1))

In [525]:
mat

<29469x362033 sparse matrix of type '<class 'numpy.int64'>'
	with 1086091 stored elements in COOrdinate format>

In [526]:
%%time
svder = TruncatedSVD(n_components=5, n_iter=30)#n_iter=40
feats = svder.fit_transform(mat)

CPU times: user 18.7 s, sys: 2.92 s, total: 21.7 s
Wall time: 6.38 s


In [562]:
d = pd.DataFrame(feats, columns=[f"group_emb_{i}" for i in range(feats.shape[1])])
d['uid'] = uids
d

,group_emb_0,group_emb_1,group_emb_2,group_emb_3,group_emb_4,uid
0,0.000000,0.000000,0.000000,0.000000,0.000000,356
1,0.841910,-0.142745,-0.358280,-0.330278,0.551151,26356
2,0.784491,-0.085189,0.162762,0.252524,-0.518312,97
3,0.854182,0.298407,-0.017655,-0.568325,0.572436,319
4,0.933955,0.186001,0.591972,-0.330767,-0.209415,30288
...,...,...,...,...,...,...
29464,0.031818,-0.057479,0.002572,0.013103,0.049308,33331
29465,0.000000,0.000000,0.000000,0.000000,0.000000,119788
29466,1.805394,-0.345865,-0.667394,-0.086383,0.492951,97581
29467,0.028815,-0.064940,0.002150,0.043474,0.064320,104038


In [581]:
train.edu

,uid,school_education,graduation_1,graduation_2,graduation_3,graduation_4,graduation_5,graduation_6,graduation_7,age,registered_year
0,356,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48,2017
1,26356,NaN,NaN,NaN,NaN,NaN,2006.0,NaN,NaN,49,2008
2,97,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35,2011
3,319,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37,2020
4,30288,2008.0,NaN,NaN,NaN,NaN,2013.0,NaN,NaN,30,2008
...,...,...,...,...,...,...,...,...,...,...,...
29464,33331,1987.0,NaN,NaN,NaN,NaN,1991.0,NaN,NaN,51,2008
29465,119788,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21,2013
29466,97581,2002.0,NaN,NaN,NaN,NaN,2007.0,NaN,NaN,36,2010
29467,104038,1969.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,69,2013


In [585]:
train.edu.set_index('uid').join(d.set_index('uid')).reset_index()

,uid,school_education,graduation_1,graduation_2,graduation_3,graduation_4,graduation_5,graduation_6,graduation_7,age,registered_year,group_emb_0,group_emb_1,group_emb_2,group_emb_3,group_emb_4
0,356,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48,2017,0.000000,0.000000,0.000000,0.000000,0.000000
1,26356,NaN,NaN,NaN,NaN,NaN,2006.0,NaN,NaN,49,2008,0.841910,-0.142745,-0.358280,-0.330278,0.551151
2,97,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35,2011,0.784491,-0.085189,0.162762,0.252524,-0.518312
3,319,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37,2020,0.854182,0.298407,-0.017655,-0.568325,0.572436
4,30288,2008.0,NaN,NaN,NaN,NaN,2013.0,NaN,NaN,30,2008,0.933955,0.186001,0.591972,-0.330767,-0.209415
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29464,33331,1987.0,NaN,NaN,NaN,NaN,1991.0,NaN,NaN,51,2008,0.031818,-0.057479,0.002572,0.013103,0.049308
29465,119788,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21,2013,0.000000,0.000000,0.000000,0.000000,0.000000
29466,97581,2002.0,NaN,NaN,NaN,NaN,2007.0,NaN,NaN,36,2010,1.805394,-0.345865,-0.667394,-0.086383,0.492951
29467,104038,1969.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,69,2013,0.028815,-0.064940,0.002150,0.043474,0.064320


In [629]:
keras.metrics.mean_squared_error

<function tensorflow.python.keras.losses.mean_squared_error(y_true, y_pred)>

In [635]:
np.nanmin

<function numpy.nanmin(a, axis=None, out=None, keepdims=<no value>)>

In [650]:
np.nanmean(train.edu.school_education)

2000.9509545753785

In [652]:
train.edu.age.max()

89

In [794]:
np.array([0,10,20,30,40])[ [2,3] ]

array([20, 30])